In [ ]:
# %load init.py
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from core import utils
import boto3
pd.set_option('display.max_columns',100);pd.set_option('display.max_rows',1000)
%matplotlib inline


In [ ]:
# Connect to AWS and see what's going on
client = boto3.client('stepfunctions', region_name='us-east-1')
base_arn = 'arn:aws:states:us-east-1:643366669028:%s:%s'
STEP_FUNCTION_ARN = base_arn % ('stateMachine', 'run_sbg_workflow_2')

executions = client.list_executions(stateMachineArn=STEP_FUNCTION_ARN, maxResults=1000)
print(len(executions.get('executions')))






In [ ]:
exec_df = pd.DataFrame(executions.get('executions'))
exec_df.sort_values(by='startDate', ascending=False, inplace=True)
exec_df.head()

In [ ]:
#clean up the names
to_replace = ['fastqc*', 'validate*', 'test*']
value = ['fastqc', 'validate', 'testing']
exec_df['type'] = exec_df.name.replace(to_replace='[-,_].*$', value='', regex=True, inplace=False)
exec_df.head()





In [ ]:
group_by_status = exec_df.groupby(['type', 'status'])
group_by_status.size().unstack().plot(kind='bar', stacked=True)


In [ ]:
#Failures in last couple of day
import datetime
cutoff = datetime.datetime.today() - datetime.timedelta(days=1) 
latest = exec_df.loc[(exec_df.startDate > cutoff) & (exec_df.status.apply(lambda x: x in ['FAILED', 'ABORTED']))]

In [ ]:
# get the url to view the result
exec_name = 'fastqc-0-11-4-1-1_4DNFIWJZJ6JS.fastq.gz'

def get_exec_arn(name):
    return latest.loc[latest.name == name, 'executionArn'].iloc[0]
    
    
def get_exec_url(name):

    #arn = latest.iloc[0]['executionArn']
    arn = latest.loc[latest.name == name, 'executionArn'].iloc[0]
    baseurl = 'https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/'
    return '%s%s' % (baseurl, arn)

url = get_exec_url(exec_name)
arn = get_exec_arn(exec_name)
print(url, arn)

In [ ]:
resp = client.get_execution_history(executionArn=arn, reverseOrder=True)

for i, event in enumerate(resp.get('events')):
    if event.get('type') == 'LambdaFunctionScheduled':
        break
failing_lambda = resp.get('events')[:i+1]
lambda_arn = failing_lambda[-1]['lambdaFunctionScheduledEventDetails']['resource']
lambda_time = failing_lambda[-1]['timestamp']
failure = failing_lambda[0].get('executionFailedEventDetails')

print("lambda %s \nRunning at %s failed \nFailure is %s" % (lambda_arn, str(lambda_time), failure))

In [ ]:
#TODO: get logGroupName and LogStreamName so I can get associated cloud watch logs